<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b431040084c717e795786ac67cdc54e76297fc39a586d404361e6bd6687301a1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:


      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 16:06:27
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: 14.50 K (0.11%)
Current PnL: -23.21 L (-15.61%)
CY Booked + Current PnL: -11.77 L (-7.92%)
-------------------
Total profit:  94.03 K
Total loss:  -24.15 L
-------------------
Total Booked + Current PnL: 15.21 L (12.5%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.39%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.08 L (65.31%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.17,36.0,M-SC,2.45,83237.0,-17540.0,17630.0,0.07,-17.40,21.18,0.09,245.0,-0.99,0.61,9.36,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189560.0,24890.0,18444.0,0.29,15.12,9.73,26.31,89.0,1.35,1.39,24.30,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108072.0,7568.0,21539.0,1.66,7.53,19.93,28.96,79.0,0.35,0.79,44.06,MH,ATH,METALS
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199145.0,-993.0,21886.0,0.53,-0.50,10.99,10.44,4.0,-0.05,1.46,5.04,X40,NTT,FMCG
84,VOLTAS,1530.00,-1.78,48.0,H-MC,2.89,206175.0,14433.0,23318.0,2.56,7.53,11.31,19.69,99.0,0.62,1.51,14.84,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,6.72,92562.0,-9648.0,103040.0,11.94,-9.44,111.32,91.37,208.0,-0.09,0.68,50.26,XY25,NTT,FINANCE
10,BANDHANBNK,400.00,-13.31,47.0,H-SC,4.17,217321.0,-61238.0,318680.0,4.62,-21.98,146.64,92.42,151.0,-0.19,1.59,25.59,XY24,NTT,BANKS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.44,124416.0,-18063.0,37586.0,4.30,-12.68,30.21,13.70,163.0,-0.48,0.91,46.85,OX40N,NTT,BANKS
56,RELAXO,1176.00,-38.14,42.0,H-SC,1.94,84318.0,-60842.0,136772.0,4.24,-41.91,162.21,52.31,136.0,-0.44,0.62,12.86,X40N,NTT,FOOTWEAR
52,PGHH,17973.08,-26.31,67.0,H-MC,3.19,213210.0,12390.0,56394.0,4.15,6.17,26.45,34.25,80.0,0.22,1.56,11.69,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-24.02,45.0,H-SC,17.26,203398.0,-67363.0,397318.0,-4.13,-24.88,195.34,121.86,133.0,-0.17,1.49,59.11,AR,ATH,IT
15,BSOFT,836.99,-20.99,29.0,H-SC,13.88,88714.0,-51939.0,124714.0,-3.67,-36.93,140.58,51.74,131.0,-0.42,0.65,1.17,XR,ATH,IT
28,HATHWAY,31.40,852.90,25.0,H-SC,2.04,82200.0,-26520.0,106202.0,-3.45,-24.39,129.20,73.29,161.0,-0.25,0.60,9.95,XR,NTT,ENTERTAINMENT
26,GREENPANEL,537.00,193.28,39.0,M-SC,0.99,130906.0,-52172.0,131155.0,-2.81,-28.50,100.19,43.14,230.0,-0.40,0.96,23.14,XY24,NTT,MISC
85,WHIRLPOOL,2270.00,-46.72,26.0,M-SC,1.30,90068.0,-1430.0,80179.0,-2.57,-1.56,89.02,86.07,223.0,-0.02,0.66,30.79,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226805.0,1843.0,165817.0,0.53,0.82,73.11,74.53,192.0,0.01,1.66,33.34,XY24,BTT,STEEL
37,INDIAMART,4850.92,-55.76,23.0,H-SC,9.96,123511.0,175.0,133589.0,-1.54,0.14,108.16,108.45,119.0,0.00,0.91,22.53,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.44,124416.0,-18063.0,37586.0,4.30,-12.68,30.21,13.70,163.0,-0.48,0.91,46.85,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,36.0,M-SC,2.45,83237.0,-17540.0,17630.0,0.07,-17.40,21.18,0.09,245.0,-0.99,0.61,9.36,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220608.0,-49059.0,85397.0,-0.59,-18.19,38.71,13.47,138.0,-0.57,1.62,11.58,XY24,NTT,PAINTS
18,CERA,9475.0,-22.20,38.0,H-SC,1.68,141646.0,-34257.0,76276.0,-0.34,-19.48,53.85,23.89,149.0,-0.45,1.04,22.33,OX40N,NTT,CERAMICS
66,SIS,528.0,2080.31,40.0,H-SC,4.02,87988.0,-23044.0,46651.0,-0.04,-20.75,53.02,21.26,156.0,-0.49,0.65,18.88,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226805.0,1843.0,165817.0,0.53,0.82,73.11,74.53,192.0,0.01,1.66,33.34,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,3.63,189560.0,24890.0,18444.0,0.29,15.12,9.73,26.31,89.0,1.35,1.39,24.30,X40N,NTT,AC
84,VOLTAS,1530.00,-1.78,48.0,H-MC,2.89,206175.0,14433.0,23318.0,2.56,7.53,11.31,19.69,99.0,0.62,1.51,14.84,XY25,NTT,AC
17,CAMS,5211.76,-6.71,42.0,H-SC,1.81,105991.0,3987.0,39937.0,-0.38,3.91,37.68,43.06,122.0,0.10,0.78,21.69,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,52.0,M-LC,7.00,153950.0,13110.0,31375.0,0.98,9.31,20.38,31.59,66.0,0.42,1.13,38.26,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108072.0,7568.0,21539.0,1.66,7.53,19.93,28.96,79.0,0.35,0.79,44.06,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,55.0,H-MC,8.45,182668.0,2860.0,109546.0,2.10,1.59,59.97,62.52,88.0,0.03,1.34,36.22,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.76,23.0,H-SC,9.96,123511.0,175.0,133589.0,-1.54,0.14,108.16,108.45,119.0,0.00,0.91,22.53,AR,ATH,MISC
32,HINDZINC,730.22,24.68,59.0,M-LC,2.96,202775.0,-2301.0,114872.0,3.80,-1.12,56.65,54.89,52.0,-0.02,1.49,22.70,X5K,ATH,METALS
85,WHIRLPOOL,2270.00,-46.72,26.0,M-SC,1.30,90068.0,-1430.0,80179.0,-2.57,-1.56,89.02,86.07,223.0,-0.02,0.66,30.79,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-6.90,26.0,H-LC,13.18,253012.0,-14353.0,79800.0,-0.56,-5.37,31.54,24.48,48.0,-0.18,1.86,46.65,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.51,206149.0,-45619.0,98828.0,-0.01,-18.12,47.94,21.13,27.0,-0.46,1.51,10.89,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272233.0,-73723.0,148585.0,-0.10,-21.31,54.58,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217822.0,-24102.0,87368.0,-0.57,-9.96,40.11,26.15,8.0,-0.28,1.60,2.93,X40,ATH,IT
19,CIPLA,1795.00,-23.55,34.0,H-LC,4.22,203856.0,-644.0,42055.0,-0.76,-0.31,20.63,20.25,10.0,-0.02,1.50,8.39,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.36,45.0,H-LC,1.61,232110.0,-14041.0,93239.0,1.64,-5.70,40.17,32.18,16.0,-0.15,1.70,28.28,X200,ATH,IT
42,ITC,452.0,-38.34,49.0,H-LC,1.89,199145.0,-993.0,21886.0,0.53,-0.50,10.99,10.44,4.0,-0.05,1.46,5.04,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.47,42.0,H-LC,2.40,155260.0,-30835.0,78313.0,0.29,-16.57,50.44,25.51,15.0,-0.39,1.14,14.41,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.41,42.0,H-LC,3.93,214157.0,3791.0,24992.0,-0.35,1.80,11.67,13.68,37.0,0.15,1.57,18.62,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.55,34.0,H-LC,4.22,203856.0,-644.0,42055.0,-0.76,-0.31,20.63,20.25,10.0,-0.02,1.50,8.39,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272233.0,-73723.0,148585.0,-0.10,-21.31,54.58,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,45.0,H-LC,7.20,225062.0,-9082.0,57706.0,0.86,-3.88,25.64,20.77,86.0,-0.16,1.65,2.77,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,217822.0,-24102.0,87368.0,-0.57,-9.96,40.11,26.15,8.0,-0.28,1.60,2.93,X40,ATH,IT
83,VBL,671.64,-20.04,35.0,H-LC,8.47,288185.0,-27657.0,140317.0,1.56,-8.76,48.69,35.67,5.0,-0.20,2.11,3.88,X40N,ATH,FMCG
40,INFY,2275.00,-21.03,38.0,H-LC,8.20,307103.0,-5785.0,177475.0,-0.49,-1.85,57.79,54.87,3.0,-0.03,2.25,4.68,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,42.0,H-LC,2.40,155260.0,-30835.0,78313.0,0.29,-16.57,50.44,25.51,15.0,-0.39,1.14,14.41,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199145.0,-993.0,21886.0,0.53,-0.50,10.99,10.44,4.0,-0.05,1.46,5.04,X40,NTT,FMCG
19,CIPLA,1795.00,-23.55,34.0,H-LC,4.22,203856.0,-644.0,42055.0,-0.76,-0.31,20.63,20.25,10.0,-0.02,1.50,8.39,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.51,206149.0,-45619.0,98828.0,-0.01,-18.12,47.94,21.13,27.0,-0.46,1.51,10.89,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,42.0,H-LC,3.93,214157.0,3791.0,24992.0,-0.35,1.80,11.67,13.68,37.0,0.15,1.57,18.62,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,28.0,H-LC,12.18,272233.0,-73723.0,148585.0,-0.10,-21.31,54.58,21.64,1.0,-0.50,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-21.03,38.0,H-LC,8.20,307103.0,-5785.0,177475.0,-0.49,-1.85,57.79,54.87,3.0,-0.03,2.25,4.68,X40,BTT,IT
42,ITC,452.00,-38.34,49.0,H-LC,1.89,199145.0,-993.0,21886.0,0.53,-0.50,10.99,10.44,4.0,-0.05,1.46,5.04,X40,NTT,FMCG
83,VBL,671.64,-20.04,35.0,H-LC,8.47,288185.0,-27657.0,140317.0,1.56,-8.76,48.69,35.67,5.0,-0.20,2.11,3.88,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,7.68,249974.0,-11645.0,130861.0,0.66,-4.45,52.35,45.57,7.0,-0.09,1.83,6.37,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,40.26,78586.0,-34963.0,75018.0,1.24,-30.79,95.46,35.27,268.0,-0.47,0.58,92.34,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.29,90060.0,-7920.0,29522.0,-0.81,-8.08,32.78,22.05,152.0,-0.27,0.66,30.91,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,6.72,92562.0,-9648.0,103040.0,11.94,-9.44,111.32,91.37,208.0,-0.09,0.68,50.26,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.86,108072.0,7568.0,21539.0,1.66,7.53,19.93,28.96,79.0,0.35,0.79,44.06,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.44,124416.0,-18063.0,37586.0,4.30,-12.68,30.21,13.70,163.0,-0.48,0.91,46.85,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1171.13,63.0,M-SC,13.93,172800.0,-8343.0,53810.0,0.90,-4.61,31.14,25.10,235.0,-0.16,1.27,38.48,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-180.68,74.0,M-SC,6.72,92562.0,-9648.0,103040.0,11.94,-9.44,111.32,91.37,208.0,-0.09,0.68,50.26,XY25,NTT,FINANCE
60,SAIL,228.00,43.54,53.0,M-MC,10.68,226805.0,1843.0,165817.0,0.53,0.82,73.11,74.53,192.0,0.01,1.66,33.34,XY24,BTT,STEEL
7,ATULAUTO,844.00,3510.81,46.0,M-SC,7.40,112525.0,-34843.0,86656.0,-0.23,-23.64,77.01,35.16,236.0,-0.40,0.83,14.93,XY24,NTT,AUTO
32,HINDZINC,730.22,24.68,59.0,M-LC,2.96,202775.0,-2301.0,114872.0,3.80,-1.12,56.65,54.89,52.0,-0.02,1.49,22.70,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.23
1,25,43.74
2,50,75.04


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    32.54
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      14.81
X40N     11.46
XR       11.07
XY25     10.25
AR        8.22
OX40N     7.89
X200      1.70
MH        1.61
X5K       1.49
SR        1.19
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.23
H-LC    25.99
H-MC    20.81
M-SC    13.03
M-LC     5.50
M-MC     2.63
L-SC     1.46
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.15,-6.33,41.43
IT,12.72,-21.16,87.22
FINANCE,10.13,-18.82,70.66
MISC,7.00,-20.74,87.87
BANKS,6.27,-16.60,77.94
PAINTS,5.63,-20.13,38.03
ELECTRICAL,5.48,-10.04,48.64
INSURANCE,3.76,-7.89,48.22
AC,3.61,1.70,14.76


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3248918.0,22
XR,1401706.0,14
AR,1212054.0,9
X40,828292.0,10
X40N,642771.0,9
OX40N,570876.0,10
XY25,441739.0,7
SR,258648.0,2
X5K,114872.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3809775.0,29
M-SC,1470494.0,17
H-LC,1276374.0,15
H-MC,1266386.0,15
M-LC,400533.0,4
M-MC,318998.0,2
L-SC,263019.0,3
L-MC,61364.0,1
L-LC,40573.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1249961.0      6
M-SC       XY24       843396.0      7
H-SC       AR         830535.0      5
           XR         816100.0      7
H-MC       XY24       582921.0      4
H-LC       X40        534142.0      5
M-MC       XY24       318998.0      2
H-SC       X40N       318007.0      4
           OX40N      263662.0      4
           SR         258648.0      2
H-LC       X40N       240078.0      3
H-MC       X40        216165.0      4
H-LC       AR         209174.0      2
H-MC       XY25       178185.0      2
M-LC       XY24       173842.0      2
L-SC       XR         173641.0      2
M-SC       AR         172345.0      2
           OX40N      144492.0      4
           XR         129236.0      2
H-LC       XY25       119941.0      3
M-LC       X5K        114872.0      1
           XR         111819.0      1
H-MC       XR         109546.0      1
M-SC       XY25       103040.0      1
H-LC       X200        93239.0      1
L-SC       OX40N       89378.0      1
H-MC       X40N        84686.0      2
H-LC       XY24        79800.0      1
M-SC       X40         77985.0      1
H-MC       OX40N       73344.0      1
H-SC       MH          72862.0      1
L-MC       XR          61364.0      1
L-LC       XY25        40573.0      1
H-MC       MH          21539.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
